In [ ]:
## https://colab.research.google.com/github/ageron/julia_notebooks/blob/master/Julia_Colab_Notebook_Template.ipynb

%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.7.1" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools Plots"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -n "$COLAB_GPU" ] && [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  if [ "$COLAB_GPU" = "1" ]; then
      JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Installing Julia 1.7.1 on the current Colab Runtime...
2022-01-12 02:02:20 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.7/julia-1.7.1-linux-x86_64.tar.gz [123374573/123374573] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
Installing Julia package BenchmarkTools...
Installing Julia package Plots...


In [1]:
versioninfo()

Julia Version 1.7.1
Commit ac5cc99908 (2021-12-22 19:35 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Xeon(R) CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-12.0.1 (ORCJIT, broadwell)
Environment:
  JULIA_NUM_THREADS = 2


In [2]:
import Pkg; Pkg.add("Arpack")
import Pkg; Pkg.add("ArgParse")
import Pkg; Pkg.add("LinearMaps")

using LinearAlgebra
using Arpack
using SparseArrays
using LinearMaps
#using ArgParse

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed Arpack_jll ─ v3.5.0+3
   Installed Arpack ───── v0.5.3
    Updating `~/.julia/environments/v1.7/Project.toml`
  [7d9fca2a] + Arpack v0.5.3
    Updating `~/.julia/environments/v1.7/Manifest.toml`
  [7d9fca2a] + Arpack v0.5.3
  [68821587] + Arpack_jll v3.5.0+3
Precompiling project...
  ✓ Arpack_jll
  ✓ Arpack
  2 dependencies successfully precompiled in 2 seconds (134 already precompiled)
   Resolving package versions...
   Installed TextWrap ─ v1.0.1
   Installed ArgParse ─ v1.1.4
    Updating `~/.julia/environments/v1.7/Project.toml`
  [c7e460c6] + ArgParse v1.1.4
    Updating `~/.julia/environments/v1.7/Manifest.toml`
  [c7e460c6] + ArgParse v1.1.4
  [b718987f] + TextWrap v1.0.1
Precompiling project...
  ✓ TextWrap
  ✓ ArgParse
  2 dependencies successfully precompiled in 1 seconds (136 already precompiled)
   Resolving package versions...
   Installed LinearMaps ─ v3.5.1
    Upda

In [3]:
function get_combination(NOS,NOD)
    combination = zeros(Int64,(NOS,NOD))
    @inbounds for j in 1:NOD
        for i in 1:NOS
            combination[i,j] = f_combination(i,j)
        end
    end
    return combination
end

get_combination (generic function with 1 method)

In [4]:
function f_combination(n,k)
    if n<k
        return 0
    end
    nCk = 1
    for i in 1:k
        nCk = nCk * (n-k+i)
        nCk = div(nCk,i)
    end
    return nCk
end

f_combination (generic function with 1 method)

In [5]:
function insertion_sort!(a,NOD)
    for i in 2:NOD
        j = i - 1
        @inbounds temp = a[i]
        @inbounds while a[j] > temp
            a[j+1] = a[j]
            j = j - 1
            if j==0
                break
            end
        end
        @inbounds a[j+1] = temp
    end
    return 0
end

insertion_sort! (generic function with 1 method)

In [6]:
function inv_list(ni,NOD,combination)
    @inbounds val_inv_list = ni[1]
    @inbounds for i in 2:NOD
        val_inv_list = val_inv_list + combination[ni[i]-1,i]
    end
    return val_inv_list
end

inv_list (generic function with 1 method)

In [7]:
function qsort_w_order!(a,o,first,last)
    @inbounds x = a[div(first+last,2)]
    i = first
    j = last
    while true
        @inbounds while a[i] < x
            i = i + 1
        end
        @inbounds while x < a[j]
            j = j - 1
        end
        if i >= j
            break
        end
        @inbounds t8 = a[i]; @inbounds a[i] = a[j]; @inbounds a[j] = t8
        @inbounds t  = o[i]; @inbounds o[i] = o[j]; @inbounds o[j] = t
        i = i + 1
        j = j - 1
    end
    if first < i - 1
        qsort_w_order!(a,o,first,i-1)
    end
    if j + 1 < last
        qsort_w_order!(a,o,j+1,last)
    end
    return 0
end

qsort_w_order! (generic function with 1 method)

In [8]:
## output "ni" is returned
function list_fly(t,NOD,NOS,combination)
    ni = zeros(Int64,NOD)
    s = t
    j = NOS - 1
    for i in NOD:-1:2
        b, j0 = binary_search(s,combination[:,i],i,j)
        j = j0 - 1
        ni[i] = j0
        s = s - combination[j,i]
    end
    ni[1] = s
    return ni
end

list_fly (generic function with 1 method)

In [9]:
## output "ni" is in arguments
function list_fly_2!(t,NOD,NOS,combination,ni)
    @inbounds fill!(ni,0)
    s = t
    j = NOS - 1
    @inbounds for i in NOD:-1:2
        b, j0 = binary_search(s,combination[:,i],i,j)
        j = j0 - 1
        ni[i] = j0
        s = s - combination[j,i]
    end
    @inbounds ni[1] = s
    return 0
end

list_fly_2! (generic function with 1 method)

In [10]:
function binary_search(s,list_s,ls,le)
    bmin = ls; bmax = le
    while true
        b = bmin + div(bmax-bmin,2)
        @inbounds if s < list_s[b]
            bmax = b - 1
        elseif list_s[b] < s
            bmin = b + 1
        else
            bmin = b
            return b, bmin
        end
        if bmin > bmax
            b = -1
            return b, bmin
        end
    end
    return b, bmin
end

binary_search (generic function with 1 method)

In [11]:
function list_to_state_no_duplication(st_list,NOS)
    string01 = ""
    for i in 1:NOS
        if i in st_list
            string01 = string01 * "1" # down
        else
            string01 = string01 * "0" # up
        end
    end
    return string01
end

list_to_state_no_duplication (generic function with 1 method)

In [12]:
function list_to_state(st_list,NOS)
    list01 = zeros(Int64,NOS)
    for i in st_list
        list01[i] += 1
    end
    list01 = mod.(list01,2)
    string01 = join(list01)
    return string01
end

list_to_state (generic function with 1 method)

In [13]:
## output "nd" is returned
function j_flip_ni(i,j,n,NOD)
    nd = ones(Int64,NOD)
    kr = NOD
    for _kr in NOD:-1:1
        if j < n[_kr]
            kr = _kr
            continue
        elseif j > n[_kr]
            kr = _kr
            break
        else
            fill!(nd,0)
            kr = _kr
            break
        end
    end
    if nd[NOD] == 1 # S+_i S-_j
        kl = 1
        for _kl in 1:kr
            if i == n[_kl]
                kl = _kl
                break
            end
            kl = _kl+1
        end
        nd[kl:kr-1] = n[kl+1:kr]
        nd[kr] = j
    else # S-_i S+_j
        kl = 1
        for _kl in 1:kr
            if i < n[_kl]
                kl = _kl
                break
            end
            kl = _kl+1
        end
        nd[kl] = i
        nd[kl+1:kr] = n[kl:kr-1]
    end
    nd[1:kl-1] = n[1:kl-1]
    nd[kr+1:NOD] = n[kr+1:NOD]
    return nd
end

j_flip_ni (generic function with 1 method)

In [14]:
## output "nd" is in arguments
function j_flip_ni_2!(i,j,n,NOD,nd)
    @inbounds fill!(nd,1)
    kr = NOD
    @inbounds for _kr in NOD:-1:1
        @inbounds if j < n[_kr]
            kr = _kr
            continue
        elseif j > n[_kr]
            kr = _kr
            break
        else
            @inbounds fill!(nd,0)
            kr = _kr
            break
        end
    end
    @inbounds if nd[NOD] == 1 # S+_i S-_j
        kl = 1
        for _kl in 1:kr
            @inbounds if i == n[_kl]
                kl = _kl
                break
            end
            kl = _kl+1
        end
        @inbounds nd[kl:kr-1] = n[kl+1:kr]
        @inbounds nd[kr] = j
    else # S-_i S+_j
        kl = 1
        for _kl in 1:kr
            @inbounds if i < n[_kl]
                kl = _kl
                break
            end
            kl = _kl+1
        end
        @inbounds nd[kl] = i
        @inbounds nd[kl+1:kr] = n[kl:kr-1]
    end
    @inbounds nd[1:kl-1] = n[1:kl-1]
    @inbounds nd[kr+1:NOD] = n[kr+1:NOD]
    return 0
end

j_flip_ni_2! (generic function with 1 method)

In [15]:
## output "Ham" is returned
function make_full_hamiltonian(lv,combination,NOD,NOxxz,p_xxz,sJint,NOS)
    Ham = zeros(Float64,(lv,lv))
    for i in 1:lv
        st_list = list_fly(i,NOD,NOS,combination)
        for j in 1:NOxxz
            f1 = p_xxz[1,j] in st_list
            f2 = p_xxz[2,j] in st_list
            if xor(f1,f2)
                Ham[i,i] = Ham[i,i] - sJint[j,2]
                ni = j_flip_ni(p_xxz[1,j],p_xxz[2,j],st_list,NOD)
                id = inv_list(ni,NOD,combination)
                Ham[i,id] = Ham[i,id] + sJint[j,1]
            else
                Ham[i,i] = Ham[i,i] + sJint[j,2]
            end
        end
    end
    return Ham
end

make_full_hamiltonian (generic function with 1 method)

In [16]:
## output "Ham" is in arguments
function make_full_hamiltonian_2!(lv,Ham,combination,NOD,NOxxz,p_xxz,sJint,NOS)
    st_list = zeros(Int64,NOD)
    ni = zeros(Int64,NOD)
    @inbounds for i in 1:lv
        list_fly_2!(i,NOD,NOS,combination,st_list)
        @inbounds for j in 1:NOxxz
            f1 = p_xxz[1,j] in st_list
            f2 = p_xxz[2,j] in st_list
            @inbounds if xor(f1,f2)
                Ham[i,i] = Ham[i,i] - sJint[j,2]
                j_flip_ni_2!(p_xxz[1,j],p_xxz[2,j],st_list,NOD,ni)
                id = inv_list(ni,NOD,combination)
                Ham[i,id] = Ham[i,id] + sJint[j,1]
            else
                Ham[i,i] = Ham[i,i] + sJint[j,2]
            end
        end
    end
    return Ham
end

make_full_hamiltonian_2! (generic function with 1 method)

In [17]:
function make_parameters_1d(NOS,NOxxz)
    p_xxz = zeros(Int64,(2,NOxxz))
    Jint = zeros(Float64,(NOxxz,2)) # Jint[NOxxz,1] --> Jint_x, Jint[NOxxz,2] --> Jint_z
    sJint = zeros(Float64,(NOxxz,2)) # sJint[NOxxz,1] --> sJint_x, sJint[NOxxz,2] --> sJint_z
    @inbounds for i in 1:NOS
        p_xxz[1,i] = mod(i-1,NOS)+1
        p_xxz[2,i] = mod(i,NOS)+1
        @inbounds if p_xxz[1,i] > p_xxz[2,i] # assume i<j for pair (i,j)
            tmp = p_xxz[1,i]
            p_xxz[1,i] = p_xxz[2,i]
            p_xxz[2,i] = tmp
        end
    end
    @inbounds fill!(Jint,1.0)
    @inbounds sJint[:,1] = 0.5 .* Jint[:,1]
    @inbounds sJint[:,2] = 0.25 .* Jint[:,2]
    return p_xxz, Jint, sJint
end

make_parameters_1d (generic function with 1 method)

In [18]:
## memory allocation within get_vec
function ham_to_vec_wave_vector!(lv,combination,NOD,NOxxz,p_xxz,sJint,NOS)
    function get_vec!(v0::AbstractVector,v1::AbstractVector) ## v0: new output, v1: old input
        length(v0) == lv || throw(DimensionMismatch())
        length(v1) == lv || throw(DimensionMismatch())
        for i = 1:lv
            v0[i] = 0.0 + 0.0im
            st_list = list_fly(i,NOD,NOS,combination)
            for j in 1:NOxxz
                f1 = p_xxz[1,j] in st_list
                f2 = p_xxz[2,j] in st_list
                if xor(f1,f2)
                    v0[i] = v0[i] - sJint[j,2] * v1[i]
                    ni = j_flip_ni(p_xxz[1,j],p_xxz[2,j],st_list,NOD)
                    id = inv_list(ni,NOD,combination)
                    v0[i] = v0[i] + sJint[j,1] * v1[id]
                else
                    v0[i] = v0[i] + sJint[j,2] * v1[i]
                end
            end
        end
        return v0
    end
    return (v0,v1) -> get_vec!(v0,v1)
end

ham_to_vec_wave_vector! (generic function with 1 method)

In [19]:
## memory allocation outside get_vec
function ham_to_vec_wave_vector_2!(lv,combination,NOD,NOxxz,p_xxz,sJint,NOS)
    st_list = zeros(Int64,NOD)
    ni = zeros(Int64,NOD)
    function get_vec_2!(v0::AbstractVector,v1::AbstractVector) ## v0: new output, v1: old input
        length(v0) == lv || throw(DimensionMismatch())
        length(v1) == lv || throw(DimensionMismatch())
        @inbounds for i = 1:lv
            v0[i] = 0.0 + 0.0im
            list_fly_2!(i,NOD,NOS,combination,st_list)
            @inbounds for j in 1:NOxxz
                f1 = p_xxz[1,j] in st_list
                f2 = p_xxz[2,j] in st_list
                @inbounds if xor(f1,f2)
                    v0[i] = v0[i] - sJint[j,2] * v1[i]
                    j_flip_ni_2!(p_xxz[1,j],p_xxz[2,j],st_list,NOD,ni)
                    id = inv_list(ni,NOD,combination)
                    v0[i] = v0[i] + sJint[j,1] * v1[id]
                else
                    v0[i] = v0[i] + sJint[j,2] * v1[i]
                end
            end
        end
        return v0
    end
    return (v0,v1) -> get_vec_2!(v0,v1)
end

ham_to_vec_wave_vector_2! (generic function with 1 method)

In [20]:
function calculate_1d(NOS,NOD)
    #NOS = 4 # number of sites
    #NOD = 2 # number of down spins
    NOxxz = NOS # number of XXZ interaction
    combination = get_combination(NOS,NOD)
    THS = combination[NOS,NOD] # total Hilbert space
    println("# NOS,NOD")
    println(NOS," ",NOD)
    #display(combination)
    println("# total Hilbert space")
    println(THS)

    p_xxz, Jint, sJint = make_parameters_1d(NOS,NOxxz)
    #println(p_xxz)
    #println(Jint)
    #println(sJint)
    #println()

    get_vec_LM_2! = ham_to_vec_wave_vector_2!(THS,combination,NOD,NOxxz,p_xxz,sJint,NOS)
    Ham = LinearMap(get_vec_LM_2!,THS;ismutating=true,issymmetric=true)
    #println(Ham)
    #println()
    ene,vec = eigs(Ham,nev=5,which=:SR)
    println("# energies")
    println(ene)
    println()
    #println("# vectors")
    #for i in 1:length(ene)
    #    println(i,vec[:,i])
    #end
    #println()
end

calculate_1d (generic function with 1 method)

In [21]:
#for NOS in [4,8,16,32,64,128] # number of sites
#for NOS in [4,8,16,32,64] # number of sites
for NOS in [4,8,16,32] # number of sites
    for NOD in [1,2,3] # number of down spins
        calculate_1d(NOS,NOD)
    end
end

# NOS,NOD
4 1
# total Hilbert space
4
# energies
[-1.0, -8.921197447996612e-18, 2.5031033493453777e-17]

# NOS,NOD
4 2
# total Hilbert space
6
# energies
[-2.0, -1.0000000000000002, -3.585624251920068e-17, 4.469288692850764e-18, 4.249538007322152e-17]

# NOS,NOD
4 3
# total Hilbert space
4
# energies
[-1.0, 1.1646420306744707e-17, 1.1472988543054442e-16]

# NOS,NOD
8 1
# total Hilbert space
8
# energies
[8.853730539606862e-17, 0.2928932188134527, 0.29289321881345315, 0.9999999999999998, 0.9999999999999999]

# NOS,NOD
8 2
# total Hilbert space
28
# energies
[-1.8019377358048378, -1.2670350983613663, -1.2670350983613659, -1.1441228056353696, -1.1441228056353687]

# NOS,NOD
8 3
# total Hilbert space
56
# energies
[-3.1284190638445804, -2.4587385088948355, -2.458738508894832, -2.1451483739207156, -2.1451483739207107]

# NOS,NOD
16 1
# total Hilbert space
16
# energies
[2.0, 2.076120467488713, 2.076120467488714, 2.292893218813452, 2.2928932188134534]

# NOS,NOD
16 2
# total Hilbert space
12

┌ Warning: Adjusting nev from 5 to 3
└ @ Arpack /root/.julia/packages/Arpack/pLziT/src/Arpack.jl:92
┌ Warning: Adjusting nev from 5 to 3
└ @ Arpack /root/.julia/packages/Arpack/pLziT/src/Arpack.jl:92
